# Tarea 4: Funciones, Modelos Personalizados y Ecuaciones Diferenciales

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist, cifar10

import numpy as np
import matplotlib.pyplot as plt

import mlflow
import dagshub

plt.style.use('default')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

np.random.seed(42)
tf.random.set_seed(42)

In [4]:
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs disponibles: {len(gpus)}")

if gpus:
    for i, gpu in enumerate(gpus):
        print(f"GPU {i}: {gpu}")
    
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Crecimiento de memoria GPU configurado")
    except RuntimeError as e:
        print(f"Error al configurar GPU: {e}")
else:
    print("No se detectó GPU. El entrenamiento usará CPU.")

GPUs disponibles: 1
GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Crecimiento de memoria GPU configurado


In [5]:
dagshub.init(repo_owner='404brainnotfound-ai', repo_name='RNA', mlflow=True)
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

Accessing as 404brainnotfound-ai

Initialized MLflow to track repo "404brainnotfound-ai/RNA"

Repository 404brainnotfound-ai/RNA initialized!

Tracking URI: https://dagshub.com/404brainnotfound-ai/RNA.mlflow


---
## Problema 1: Capa RGB a Escala de Grises

In [6]:
class CapaGrises(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CapaGrises, self).__init__(**kwargs)
        self.rgb_weights = tf.constant([0.299, 0.587, 0.114], dtype=tf.float32)
    
    def call(self, inputs):
        r = inputs[:, :, :, 0]
        g = inputs[:, :, :, 1]
        b = inputs[:, :, :, 2]
        grises = 0.299 * r + 0.587 * g + 0.114 * b
        grises = tf.expand_dims(grises, axis=-1)
        return grises
    
    def get_config(self):
        config = super(CapaGrises, self).get_config()
        return config